<a href="https://colab.research.google.com/github/claudelepere/ML_GitHub/blob/main/Copy_of_BERT_for_multi_label_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## IMPORTANT

Process any ipynb file only with the editor in which it was created, don't open a JupyterLab ipynb with Colab and vice versa, don't open any ipynb with an editor that can render the display of a notebook, like VS Code or VSCodium.

Why? Because any ipynb contains metadata specific to each notebook editor and another editor may modify them.

---

Do not use conda in Colab notebook, do not try to install with conda, use pip.

---

Do not launch wandb (cannot connect to the W&B server): 3 things: #!pip install, #import, os.environ disable

In [ ]:
!pip install -q accelerate
!pip install -q huggingface_hub
!pip install -q scikit-learn
!pip install -q transformers datasets
!pip install -q wandb

import json
import numpy as np
import os
import sys
import time
import torch
import wandb

from datasets              import DatasetDict
from google.colab          import auth, drive, files, userdata
from huggingface_hub       import create_repo, login, upload_file
from huggingface_hub.utils import RepositoryNotFoundError
from sklearn.metrics       import accuracy_score, average_precision_score, classification_report, f1_score, precision_score, recall_score, roc_auc_score
from torch.utils.data      import DataLoader
from tqdm.auto             import tqdm
from transformers          import AutoModelForSequenceClassification, AutoTokenizer, EvalPrediction, Trainer, TrainingArguments

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
"""
# Check the Python version
print(sys.version)
print()

# Get the installed packages (you can see that conda is not installed (do not install it))
!pip list
print()

# Check system information
!cat /etc/os-release
!uname -m
print()

# Check the GPU details (only if the runtime type is T4 GPU)
#!nvidia-smi
#print()

# Check RAM
!free -h
print()

# Check disk space
!df -h
print()

# Get environment variables
for key, value in os.environ.items():
    print(f"{key}: {value}")
"""

print(f"currentdir: {os.getcwd()}")
print()

# Check the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"device: {device}")

currentdir: /content

device: cuda


In [ ]:
# Google Colab authenticate

# Required for accessing Colab Secrets: not necessary for Hugging Face Hub, for wandb, you still have to enter manually
#auth.authenticate_user()

In [ ]:
# Hugging Face authenticate

os.environ["HF_TOKEN"] = userdata.get("HF_TOKEN")    # Store the key in os.environ
hf_token               = os.environ.get('HF_TOKEN')
login(token=hf_token)

# Verify
!huggingface-cli whoami

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


claudelepere


In [ ]:
# Create the skill_classification repo on the Hugging Face Hub

HF_name         = "claudelepere/skill_classification"
repo_id_model   = HF_name
repo_id_dataset = HF_name

repo_model_url = create_repo(
    repo_id   = repo_id_model,
    repo_type = "model",
    private   = True,
    exist_ok  = True
    )
print(f"Repo model url: {repo_model_url} created successfully as a private repo.")

repo_dataset_url = create_repo(
    repo_id   = repo_id_dataset,
    repo_type = "dataset",
    private   = True,
    exist_ok  = True
    )
print(f"Repo datasets url: {repo_dataset_url} created successfully as a private repo.")

repo_id_dataset = f"datasets/{HF_name}"

print(f"repo_id_model: {repo_id_model}")
print(f"repo_id_dataset: {repo_id_dataset}")

Repo model url: https://huggingface.co/claudelepere/skill_classification created successfully as a private repo.
Repo datasets url: https://huggingface.co/datasets/claudelepere/skill_classification created successfully as a private repo.
repo_id_model: claudelepere/skill_classification
repo_id_dataset: datasets/claudelepere/skill_classification


In [ ]:
# Weights & Biases (W&B, wandb) authenticate

#os.environ["WANDB_DISABLE_CODE"] = "true"
os.environ["WANDB_API_KEY"] = userdata.get("WANDB_API_KEY")        # Store the key in os.environ
wandb_api_key               = os.environ.get('WANDB_API_KEY')

wandb.login(relogin=True)                                          # Force relogin

try:
  wandb.init(
      project="skill_classification",
      #entity="claudelepere",
      entity = "claudelepere-c-cile-cy"
      )
except wandb.errors.CommError as e:
  print(f"CommError: {e}")
except Exception as e:
  print(f"Exception: {e}")

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: claudelepere (claudelepere-c-cile-cy). Use `wandb login --relogin` to force relogin


# My fine-tuning BERT (and friends) for multi-label text classification

In this notebook, we are going to fine-tune BERT to predict one or more labels for a given piece of text. Note that this notebook illustrates how to fine-tune a bert-base-uncased model, but you can also fine-tune a RoBERTa, DeBERTa, DistilBERT, CANINE, ... checkpoint in the same way.

All of those work in the same way: they add a **linear layer on top of the base model, which is used to produce a tensor of shape (batch_size, num_labels)**, indicating the unnormalized scores for a number of labels for every example in the batch.



## Set-up environment

First, we install the libraries which we'll use: HuggingFace Transformers and Datasets.

## Load dataset

Next, let's download a multi-label text classification dataset from the [hub](https://huggingface.co/).

At the time of writing, I picked a random one as follows:   

* first, go to the "datasets" tab on huggingface.co
* next, select the "multi-label-classification" tag on the left as well as the the "1k<10k" tag (fo find a relatively small dataset).

Note that you can also easily load your local data (i.e. csv files, txt files, Parquet files, JSON, ...) as explained [here](https://huggingface.co/docs/datasets/loading.html#local-and-remote-files).



In [ ]:
# Upload and unzip the dataset zip file

uploaded_files     = files.upload()
uploaded_file_name = list(uploaded_files.keys())[0]
print(f"uploaded_file_name: {uploaded_file_name} {len(uploaded_files)}")

!unzip {uploaded_file_name}

unzipped_file_dir_name = os.path.splitext(uploaded_file_name)[0]
print(f"unzipped_file_dir_name: {unzipped_file_dir_name}")

Saving dataset_11_1000.zip to dataset_11_1000.zip
uploaded_file_name: dataset_11_1000.zip 1
Archive:  dataset_11_1000.zip
  inflating: dataset_11_1000/dataset_dict.json  
  inflating: dataset_11_1000/test/data-00000-of-00001.arrow  
  inflating: dataset_11_1000/test/dataset_info.json  
  inflating: dataset_11_1000/test/state.json  
  inflating: dataset_11_1000/train/data-00000-of-00001.arrow  
  inflating: dataset_11_1000/train/dataset_info.json  
  inflating: dataset_11_1000/train/state.json  
  inflating: dataset_11_1000/validation/data-00000-of-00001.arrow  
  inflating: dataset_11_1000/validation/dataset_info.json  
  inflating: dataset_11_1000/validation/state.json  
unzipped_file_dir_name: dataset_11_1000


In [ ]:
# Create the dataset: 3 Hugging Face Dataset in a Hugging Face DatasetDict

dataset = DatasetDict.load_from_disk(unzipped_file_dir_name)

print(f"dataset: {type(dataset)} {dataset.shape}\n{dataset}")

dataset: <class 'datasets.dataset_dict.DatasetDict'> {'train': (800, 8), 'validation': (50, 8), 'test': (150, 8)}
DatasetDict({
    train: Dataset({
        features: ['id', 'text', '390', '135', '136', '137', '138', '139'],
        num_rows: 800
    })
    validation: Dataset({
        features: ['id', 'text', '390', '135', '136', '137', '138', '139'],
        num_rows: 50
    })
    test: Dataset({
        features: ['id', 'text', '390', '135', '136', '137', '138', '139'],
        num_rows: 150
    })
})


As we can see, the dataset contains 3 splits: one for training, one for validation and one for testing.

Let's test the first example of the training split:

In [ ]:
example = dataset['train'][0]
print(f"example: {type(example)} {example.keys()}\n{example}")

example: <class 'dict'> dict_keys(['id', 'text', '390', '135', '136', '137', '138', '139'])
{'id': 305473, 'text': 'Talencia Consulting - Service Delivery Coordinator Power BI, Jira, MS 365 Talencia Consulting Talencia Consulting is currently looking for a Service Delivery Coordinator for one of our renowned clients based in Brussels. Start date: 01/07/24 Duration: Mid to long term assignment Homeworking: Hybrid Mission Monitor and report performance of support partners and propose improvements and solutions to incidents Run govenance and reporting meetings Manage daily escalation with vendors Carry out financial tasks in systems Requirements Bachelor or Masters degree or equivalent by experience At least 1 year of experience in a similar position (service delivery) Knowledge of working with IT Support partners/processes Knowledge of MS 365 applications, Power BI and Jira Fluent in English', '390': False, '135': False, '136': True, '137': True, '138': True, '139': False}


In [ ]:
# Create the label list and the id2label and label2id mappings.

"""
labels
    if dataset 7_1000_125_125  , 48 labels
    if dataset 7_128_18_54     , 42 labels
    if dataset 8910_1087_68_204, 206 labels
"""

labels = [label for label in dataset['train'].features.keys() if label not in ['id', 'text']]
labels.sort()
print(f"labels: {type(labels)} {len(labels)}\n{labels}")

id2label = {idx:label for idx, label in enumerate(labels)}
print(f"id2label: {type(id2label)} {len(id2label)}\n{id2label}")

label2id = {label:idx for idx, label in enumerate(labels)}
print(f"label2id: {type(label2id)} {len(label2id)}\n{label2id}")

labels: <class 'list'> 6
['135', '136', '137', '138', '139', '390']
id2label: <class 'dict'> 6
{0: '135', 1: '136', 2: '137', 3: '138', 4: '139', 5: '390'}
label2id: <class 'dict'> 6
{'135': 0, '136': 1, '137': 2, '138': 3, '139': 4, '390': 5}


The dataset consists of texts, labeled with one or more skills.

Let's create a list that contains the labels, as well as 2 dictionaries that map labels to integers and back.

In [ ]:
# Upload the label list as a JSON file to the HF repo_id_dataset

labels_path = "labels.json"
with open(labels_path, 'w') as f:
    json.dump(labels, f)
print(f"labels saved to {labels_path}")

upload_file(
    path_or_fileobj = labels_path,
    path_in_repo    = labels_path,
    repo_id         = HF_name,
    repo_type       = "dataset"
    )
print(f"labels uploaded to https://huggingface.co/datasets/{HF_name}/tree/main/{labels_path}")

labels saved to labels.json


No files have been modified since last commit. Skipping to prevent empty commit.


labels uploaded to https://huggingface.co/datasets/claudelepere/skill_classification/tree/main/labels.json


## Preprocess data

As models like BERT don't expect text as direct input, but rather **`input_ids`**, etc., we tokenize the text using the tokenizer. Here I'm using the `AutoTokenizer` API, which will automatically load the appropriate tokenizer based on the checkpoint on the hub.

What's a bit tricky is that we also need to provide labels to the model. For multi-label text classification, this is a **matrix of shape (batch_size, num_labels)**. Also important: this should be a tensor of floats rather than integers, otherwise PyTorch' **BCEWithLogitsLoss** (which the model will use) will complain, as explained [here](https://discuss.pytorch.org/t/multi-label-binary-classification-result-type-float-cant-be-cast-to-the-desired-output-type-long/117915/3).

### Preprocess (examples, not example, because batched=True => examples is a batch)

In [ ]:
# Tokenize 'text' in the 3 datasets, train, validation and test

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
def preprocess_data(examples, indices):
  text = examples['text']    # Batch of texts

  encoding = tokenizer(
      text,                             # Tokenize text
      truncation     = True,
      padding        = 'max_length',
      max_length     = 512,
      return_tensors = 'pt'             # Return PyTorch tensors
      )

  # Create an empty label matrix
  labels_matrix = torch.zeros((len(text), len(labels)), dtype=torch.float32)
  #print(f"labels_matrix: {type(labels_matrix)} {labels_matrix.shape}")

  # Populate label matrix
  for idx, label in enumerate(labels):
    #print(f"idx:{idx} label:{label}")
    if label in examples:
      labels_matrix[:, idx] = torch.tensor(
          [1.0 if val else 0.0 for val in examples[label]],
          dtype=torch.float32
          )
  #print(f"labels_matrix: {type(labels_matrix)} {labels_matrix.shape}")

  # Add labels to the encoding
  encoding['labels'] = labels_matrix
  #print(f"encoding['labels']: {encoding['labels']}")

  return encoding

In [ ]:
# Create the 3 encoded datasets, train, validation and test

encoded_dataset = dataset.map(
    preprocess_data,
    batched        = True,
    remove_columns = dataset['train'].column_names,
    with_indices   = True
    )
train_dataset      = encoded_dataset['train']
validation_dataset = encoded_dataset['validation']
test_dataset       = encoded_dataset['test']
print(f"encoded_dataset: {type(encoded_dataset)} {encoded_dataset.shape}\n{encoded_dataset}")
print(f"train_dataset: {type(train_dataset)} {train_dataset.shape}\n{train_dataset}")
print(f"validation_dataset: {type(validation_dataset)} {validation_dataset.shape}\n{validation_dataset}")
print(f"test_dataset: {type(test_dataset)} {test_dataset.shape}\n{test_dataset}")

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

Map:   0%|          | 0/150 [00:00<?, ? examples/s]

encoded_dataset: <class 'datasets.dataset_dict.DatasetDict'> {'train': (800, 4), 'validation': (50, 4), 'test': (150, 4)}
DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 800
    })
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 50
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 150
    })
})
train_dataset: <class 'datasets.arrow_dataset.Dataset'> (800, 4)
Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 800
})
validation_dataset: <class 'datasets.arrow_dataset.Dataset'> (50, 4)
Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 50
})
test_dataset: <class 'datasets.arrow_dataset.Dataset'> (150, 4)
Dataset({
    features: ['input_ids', 'token_type_ids', 'atten

In [ ]:
example = encoded_dataset['train'][0]
print(f"example: {type(example)} {example.keys()}\n{example}")

print()

print(f"example['input_ids']: {type(example['input_ids'])} {len(example['input_ids'])}\n{example['input_ids']}")
print(f"example['token_type_ids']: {type(example['token_type_ids'])} {len(example['token_type_ids'])}\n{example['token_type_ids']}")
print(f"example['attention_mask']: {type(example['attention_mask'])} {len(example['attention_mask'])}\n{example['attention_mask']}")
print(f"example['labels']:  {type(example['labels'])} {len(example['labels'])}\n{example['labels']}")

example: <class 'dict'> dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])
{'input_ids': [101, 6925, 22750, 10552, 1011, 2326, 6959, 10669, 2373, 12170, 1010, 10147, 2527, 1010, 5796, 19342, 6925, 22750, 10552, 6925, 22750, 10552, 2003, 2747, 2559, 2005, 1037, 2326, 6959, 10669, 2005, 2028, 1997, 2256, 8228, 7846, 2241, 1999, 9371, 1012, 2707, 3058, 1024, 5890, 1013, 5718, 1013, 2484, 9367, 1024, 3054, 2000, 2146, 2744, 8775, 19453, 2075, 1024, 8893, 3260, 8080, 1998, 3189, 2836, 1997, 2490, 5826, 1998, 16599, 8377, 1998, 7300, 2000, 10444, 2448, 18079, 8189, 5897, 1998, 7316, 6295, 6133, 3679, 9686, 25015, 3508, 2007, 17088, 4287, 2041, 3361, 8518, 1999, 3001, 5918, 5065, 2030, 5972, 3014, 2030, 5662, 2011, 3325, 2012, 2560, 1015, 2095, 1997, 3325, 1999, 1037, 2714, 2597, 1006, 2326, 6959, 1007, 3716, 1997, 2551, 2007, 2009, 2490, 5826, 1013, 6194, 3716, 1997, 5796, 19342, 5097, 1010, 2373, 12170, 1998, 10147, 2527, 19376, 1999, 2394, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [ ]:
tokenizer.decode(example['input_ids'])

'[CLS] talencia consulting - service delivery coordinator power bi, jira, ms 365 talencia consulting talencia consulting is currently looking for a service delivery coordinator for one of our renowned clients based in brussels. start date : 01 / 07 / 24 duration : mid to long term assignment homeworking : hybrid mission monitor and report performance of support partners and propose improvements and solutions to incidents run govenance and reporting meetings manage daily escalation with vendors carry out financial tasks in systems requirements bachelor or masters degree or equivalent by experience at least 1 year of experience in a similar position ( service delivery ) knowledge of working with it support partners / processes knowledge of ms 365 applications, power bi and jira fluent in english [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PA

In [ ]:
[id2label[idx] for idx, label in enumerate(example['labels']) if label == 1.0]

['136', '137', '138']

Finally, we set the format of our data to PyTorch tensors. This will turn the training, validation and test sets into standard PyTorch [datasets](https://pytorch.org/docs/stable/data.html).

In [ ]:
# Set PyTorch format to ensures correctness and compatibility with PyTorch pipelines

# The 3 Hugging Face Dataset are formatted as PyTorch Dataset
encoded_dataset.set_format('torch')

## Define model

Here we define a **model that includes a pre-trained base (i.e. the weights from bert-base-uncased) are loaded, with a random initialized classification head (linear layer) on top**. One should fine-tune this head, together with the pre-trained base on a labeled dataset.

This is also printed by the warning.

We set the `problem_type` to be "multi_label_classification", as this will make sure the appropriate loss function is used (namely [**BCEWithLogitsLoss**](https://pytorch.org/docs/stable/generated/torch.nn.BCEWithLogitsLoss.html)). We also make sure the output layer has `len(labels)` output neurons, and we set the id2label and label2id mappings.

In [ ]:
# Define the model

model = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    problem_type="multi_label_classification",
    num_labels=len(labels),
    id2label=id2label,
    label2id=label2id
    )

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Train the model!

We are going to train the model using HuggingFace's Trainer API. This requires us to define 2 things:

* `TrainingArguments`, which specify training hyperparameters. All options can be found in the [docs](https://huggingface.co/transformers/main_classes/trainer.html#trainingarguments). Below, we for example specify that we want to evaluate after every epoch of training, we would like to save the model every epoch, we set the learning rate, the batch size to use for training/evaluation, how many epochs to train for, and so on.
* a `Trainer` object (docs can be found [here](https://huggingface.co/transformers/main_classes/trainer.html#id1)).

In [ ]:
batch_size  = 8     # 8, 16, 32, 64, 128
metric_name = "f1"

### TrainingArguments

In [ ]:
output_dir = "training_results"  # where model predictions and checkpoints will be written during training
args = TrainingArguments(
    output_dir                  = output_dir,
    overwrite_output_dir        = True,
    logging_dir                 = "logs",
    logging_steps               = 50,
    save_steps                  = 500,
    save_total_limit            = 2,
    eval_strategy               = "epoch",
    save_strategy               = "epoch",
    learning_rate               = 2e-5,
    per_device_train_batch_size = batch_size,
    per_device_eval_batch_size  = batch_size,
    num_train_epochs            = 5,
    weight_decay                = 0.01,
    load_best_model_at_end      = True,
    metric_for_best_model       = metric_name,
    #push_to_hub                 = True,             # push the model to the HF Hub
    run_name                   = "BERT-multilabel-lr2e5-epochs5-dataset_11_50000",
    report_to                  = "none" #"wandb"
    )

We are also going to compute metrics while training. For this, we need to define a `compute_metrics` function, that returns a dictionary with the desired metric values.

In [ ]:
# Metrics
#   source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/

def multi_label_metrics(predictions, labels, threshold=0.2):
    _average = 'micro'    # 'micro' or 'weighted'

    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs   = sigmoid(torch.Tensor(predictions))

    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1

    # finally, compute metrics
    y_true               = labels
    f1                   = f1_score               (y_true=y_true, y_pred=y_pred, average=_average)    #, zero_division=1)
    precision            = precision_score        (y_true=y_true, y_pred=y_pred, average=_average)    #, zero_division=1)
    recall               = recall_score           (y_true=y_true, y_pred=y_pred, average=_average)    #, zero_division=1)
    roc_auc              = roc_auc_score          (y_true=y_true, y_score=probs, average=_average)
    precision_recall_auc = average_precision_score(y_true=y_true, y_score=probs, average=_average)
    accuracy             = accuracy_score         (y_true=y_true, y_pred=y_pred)

    # return as dictionary
    metrics = {
        'f1'                  : f1,
        'precision'           : precision,
        'recall'              : recall,
        'roc_auc'             : roc_auc,
        'precision_recall_auc': precision_recall_auc,
        'accuracy'            : accuracy
        }

    return metrics

In [ ]:
def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    result = multi_label_metrics(
        predictions = preds,
        labels      = p.label_ids
        )
    return result

Let's verify a batch as well as a forward pass:

In [ ]:
print(f"inputids:        {type(encoded_dataset['train']['input_ids'][0])}\t{encoded_dataset['train']['input_ids'][0].shape}")
print(f"token_type_ids': {type(encoded_dataset['train']['token_type_ids'][0])}\t{encoded_dataset['train']['token_type_ids'][0].shape}")
print(f"attention_mask:  {type(encoded_dataset['train']['attention_mask'][0])}\t{encoded_dataset['train']['attention_mask'][0].shape}")
print(f"labels:          {type(encoded_dataset['train'][0]['labels'])}\t{encoded_dataset['train'][0]['labels'].shape}")

inputids:        <class 'torch.Tensor'>	torch.Size([512])
token_type_ids': <class 'torch.Tensor'>	torch.Size([512])
attention_mask:  <class 'torch.Tensor'>	torch.Size([512])
labels:          <class 'torch.Tensor'>	torch.Size([6])


In [ ]:
# Execute the forward pass

outputs = model(
    input_ids      = encoded_dataset['train']['input_ids'][0].unsqueeze(0),
    attention_mask = encoded_dataset['train']['attention_mask'][0].unsqueeze(0),
    labels         = encoded_dataset['train'][0]['labels'].unsqueeze(0)
    )

print(f"outputs: {type(outputs)} {outputs.keys()}\n{outputs}")

outputs: <class 'transformers.modeling_outputs.SequenceClassifierOutput'> odict_keys(['loss', 'logits'])
SequenceClassifierOutput(loss=tensor(0.7090, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>), logits=tensor([[ 0.0205,  0.7373, -0.0674, -0.3439,  0.1976,  0.1210]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)


Let's start training!

In [ ]:
## Create the trainer

trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
    )


<ipython-input-26-485be3f189a5>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
# Train, save the results as a JSON file

train_output  = trainer.train()

train_results = {
    'global_step':   train_output.global_step,    # total steps completed during training
    'training_loss': train_output.training_loss,  # average loss during training
    'metrics':       train_output.metrics         # dictionary of metrics
}

with open("train_results.json", "w") as f:
  json.dump(train_results, f, indent=4)

Epoch,Training Loss,Validation Loss,F1,Precision,Recall,Roc Auc,Precision Recall Auc,Accuracy
1,0.378400,0.457537,0.747082,0.640000,0.897196,0.861944,0.748700,0.160000
2,0.329100,0.436201,0.741313,0.631579,0.897196,0.874582,0.777414,0.160000
3,0.296500,0.460146,0.744186,0.635762,0.897196,0.868820,0.753040,0.140000
4,0.291500,0.438139,0.750958,0.636364,0.915888,0.877294,0.746916,0.160000
5,0.257700,0.428851,0.745247,0.628205,0.915888,0.881313,0.765394,0.160000


In [ ]:
print("Training successfully completed.")

Training successfully completed.


## Evaluate

After training, we evaluate our model on the validation set.

In [ ]:
def get_results(model, dataset, batch_size, threshold):

  # Set the model to evaluation mode to disable dropout and other training-specific behaviors
  model.eval()

  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  model.to(device)

  test_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

  all_preds       = []
  all_probs       = []
  all_true_labels = []
  threshold       = threshold

  for batch in tqdm(test_loader):
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
      outputs = model(**batch)
    logits = outputs.logits

    # Convert logits to probabilities and probabilities to predictions
    sigmoid = torch.nn.Sigmoid()
    probs   = sigmoid(logits).cpu().numpy()    # Convert to Numpy
    preds   = (probs > threshold).astype(int)  # Convert to binary Numpy array

    # Accumulate probabilities, predictions and labels
    all_probs.append(probs)
    all_preds.append(preds)
    all_true_labels.append(batch['labels'].cpu().numpy())

  # Concatenate results from all batches
  all_probs       = np.concatenate(all_probs, axis=0)        # shape: [num_samples, num_labels]
  all_preds       = np.concatenate(all_preds, axis=0)        # shape: [num_samples, num_labels]
  all_true_labels = np.concatenate(all_true_labels, axis=0)  # shape: [num_samples, num_labels]

  #np.set_printoptions(threshold=np.inf)
  print(f"all_probs:       {type(all_probs)} {all_probs.shape} {all_probs}")
  print(f"all_preds:       {type(all_preds)} {all_preds.shape} {all_preds}")
  print(f"all_true_labels: {type(all_true_labels)} {all_true_labels.shape} {all_true_labels}")

  # Classification report for precision, recall, F1 score
  print(classification_report(
      y_true=all_true_labels,
      y_pred=all_preds,
      target_names=labels,
      zero_division=0
      ))

  # ROC AUC for multi-label classification
  roc_auc = roc_auc_score(
      y_true=all_true_labels,
      y_score=all_probs,
      average='micro'
      )
  print(f"ROC AUC: {roc_auc}")

In [ ]:
get_results(model=model, dataset=validation_dataset, batch_size=batch_size, threshold=0.5)

  0%|          | 0/7 [00:00<?, ?it/s]

all_probs:       <class 'numpy.ndarray'> (50, 6) [[0.01926517 0.05658763 0.3534019  0.9092309  0.90571994 0.02467113]
 [0.01568509 0.07252771 0.63612103 0.9370108  0.8600063  0.01859948]
 [0.05312638 0.17269275 0.8337916  0.90857    0.52159464 0.04752291]
 [0.01966699 0.08030602 0.682893   0.9422043  0.75966704 0.02068998]
 [0.02514493 0.0797974  0.40184078 0.87731975 0.91151214 0.0327083 ]
 [0.01933142 0.07669125 0.72402173 0.94198847 0.7875524  0.02266008]
 [0.09504924 0.21733019 0.85235626 0.87145144 0.39127743 0.07616168]
 [0.02367029 0.10950309 0.6136318  0.89124846 0.9050795  0.02794453]
 [0.01608497 0.05715619 0.48380485 0.9183389  0.910112   0.02157485]
 [0.02095459 0.06303231 0.3814158  0.89379066 0.90913266 0.03258411]
 [0.01765427 0.05281631 0.52453643 0.94012064 0.8523433  0.02099016]
 [0.02288221 0.05583001 0.29731065 0.8976232  0.90066963 0.02923072]
 [0.09512149 0.19149683 0.8319822  0.8897095  0.3341715  0.0699048 ]
 [0.02184024 0.11911456 0.81396323 0.93026257 0.782838

In [ ]:
# Evaluate, save the output as a JSON file

eval_output = trainer.evaluate()

with open("eval_results.json", "w") as f:
  json.dump(eval_output, f, indent=4)

In [ ]:
print("Evaluation successfully completed.")

Evaluation successfully completed.


In [ ]:
# Save the model

model_path = "model"
trainer.save_model(model_path)

In [ ]:
# Upload the model and the tokenizer to the HF repo_id_model

model     = AutoModelForSequenceClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

model.push_to_hub(repo_id_model)
tokenizer.push_to_hub(repo_id_model)

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/claudelepere/skill_classification/commit/d7bbb7c9462d11fed705ce7b303461356c674e10', commit_message='Upload tokenizer', commit_description='', oid='d7bbb7c9462d11fed705ce7b303461356c674e10', pr_url=None, repo_url=RepoUrl('https://huggingface.co/claudelepere/skill_classification', endpoint='https://huggingface.co', repo_type='model', repo_id='claudelepere/skill_classification'), pr_revision=None, pr_num=None)

In [ ]:
# Upload train_results.json and eval_results.json to the HF repo_id_dataset
#   BETTER to upload to wanddb repo?

upload_file(
    path_or_fileobj = "train_results.json",
    path_in_repo    = "train_results.json",
    repo_id         = HF_name,
    repo_type       = "dataset"
    )

upload_file(
    path_or_fileobj = "eval_results.json",
    path_in_repo    = "eval_results.json",
    repo_id         = HF_name,
    repo_type       = "dataset"
    )

CommitInfo(commit_url='https://huggingface.co/datasets/claudelepere/skill_classification/commit/257a0d056596bf89e81f9d829ae119bbeab4df0b', commit_message='Upload eval_results.json with huggingface_hub', commit_description='', oid='257a0d056596bf89e81f9d829ae119bbeab4df0b', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/claudelepere/skill_classification', endpoint='https://huggingface.co', repo_type='dataset', repo_id='claudelepere/skill_classification'), pr_revision=None, pr_num=None)

## Predict results on the test dataset
Faire la même chose pour l'évaluation?

# Set the model to evaluation mode to disable dropout and other training-specific behaviors

model.eval()

# Predict results on the test dataset

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

all_preds       = []
all_probs       = []
all_true_labels = []
threshold       = 0.5

for batch in tqdm(test_loader):
  batch = {k: v.to(device) for k, v in batch.items()}
  with torch.no_grad():
    outputs = model(**batch)
  logits = outputs.logits

  # Convert logits to probabilities and probabilities to predictions
  sigmoid = torch.nn.Sigmoid()
  probs   = sigmoid(logits).cpu().numpy()    # Convert to Numpy
  preds   = (probs > threshold).astype(int)  # Convert to binary Numpy array

  # Accumulate probabilities, predictions and labels
  all_probs.append(probs)
  all_preds.append(preds)
  all_true_labels.append(batch['labels'].cpu().numpy())

# Concatenate results from all batches
all_probs       = np.concatenate(all_probs, axis=0)        # shape: [num_samples, num_labels]
all_preds       = np.concatenate(all_preds, axis=0)        # shape: [num_samples, num_labels]
all_true_labels = np.concatenate(all_true_labels, axis=0)  # shape: [num_samples, num_labels]

#np.set_printoptions(threshold=np.inf)
print(f"all_probs:       {type(all_probs)} {all_probs.shape} {all_probs}")
print(f"all_preds:       {type(all_preds)} {all_preds.shape} {all_preds}")
print(f"all_true_labels: {type(all_true_labels)} {all_true_labels.shape} {all_true_labels}")

# Classification report for precision, recall, F1 score
print(classification_report(
    y_true=all_true_labels,
    y_pred=all_preds,
    target_names=labels,
    zero_division=0
    ))

# ROC AUC for multi-label classification
roc_auc = roc_auc_score(
    y_true=all_true_labels,
    y_score=all_probs,
    average='micro'
    )
print(f"ROC AUC: {roc_auc}")

In [ ]:
get_results(model=model, dataset=test_dataset, batch_size=batch_size, threshold=0.5)

  0%|          | 0/19 [00:00<?, ?it/s]

all_probs:       <class 'numpy.ndarray'> (150, 6) [[0.01691944 0.06343879 0.45983404 0.9125288  0.9115353  0.0216679 ]
 [0.01939093 0.0902111  0.6329766  0.9098724  0.8962904  0.02589396]
 [0.02395096 0.06109112 0.5616562  0.93846446 0.7486289  0.02571057]
 [0.02125109 0.05281711 0.3215841  0.91167957 0.8867488  0.02797784]
 [0.02190776 0.06987847 0.6721681  0.94407314 0.7348298  0.02239533]
 [0.02442293 0.07266927 0.3594999  0.86369133 0.926941   0.03467226]
 [0.02153329 0.05523455 0.3153904  0.8969699  0.9111211  0.0289594 ]
 [0.02100455 0.05322331 0.35161167 0.9193279  0.88088405 0.02642533]
 [0.01627562 0.05770493 0.5198076  0.9289143  0.8844083  0.02035978]
 [0.0275875  0.10133164 0.80005205 0.93946093 0.6489359  0.02806954]
 [0.02930978 0.0714993  0.28551134 0.8454764  0.9163346  0.04329865]
 [0.02000173 0.05102149 0.4141267  0.92768127 0.87613875 0.02556037]
 [0.02002436 0.09432338 0.7111809  0.93167573 0.8138822  0.02125776]
 [0.02665894 0.09358723 0.6935285  0.93251413 0.69438

In [ ]:
predictions = trainer.predict(validation_dataset)
print(predictions.predictions)  # Model logits
print(predictions.label_ids)    # Ground truth labels
print(predictions.metrics)      # Metrics

[[-3.9300032  -2.813713   -0.60411906  2.3042805   2.2624607  -3.677141  ]
 [-4.139235   -2.548494    0.55856746  2.6997325   1.8153422  -3.9658468 ]
 [-2.880492   -1.5666622   1.6127412   2.296298    0.08643238 -2.997854  ]
 [-3.9089506  -2.4381964   0.7670989   2.7913086   1.1508548  -3.857199  ]
 [-3.6576326  -2.445103   -0.39780086  1.9672904   2.3322394  -3.386871  ]
 [-3.926503   -2.488176    0.9644994   2.7873516   1.3102345  -3.7642298 ]
 [-2.2534854  -1.281293    1.7532023   1.913853   -0.44194567 -2.4956787 ]
 [-3.7195795  -2.0958269   0.46260437  2.103557    2.2549834  -3.5491915 ]
 [-4.113654   -2.803113   -0.06480327  2.4199884   2.3150036  -3.8144162 ]
 [-3.8442202  -2.6990013  -0.48354307  2.1300595   2.30309    -3.3908038 ]
 [-4.0189657  -2.886673    0.09822462  2.7536755   1.7530993  -3.842488  ]
 [-3.7542477  -2.8279946  -0.8601374   2.1710901   2.2046874  -3.5028687 ]
 [-2.2526457  -1.4403132   1.5997412   2.0877771  -0.68937784 -2.5881526 ]
 [-3.801919   -2.0008419 

In [ ]:
print("Predictions 1 successfully completed.")

Predictions 1 successfully completed.


### Or otherwise

In [ ]:
predictions = trainer.predict(test_dataset)
print(predictions.predictions)  # Model logits
print(predictions.label_ids)    # Ground truth labels
print(predictions.metrics)      # Metrics

[[-4.0622277  -2.6921394  -0.1610108   2.3449094   2.3325267  -3.8100176 ]
 [-3.9233685  -2.3110602   0.5450079   2.312078    2.1566703  -3.6275105 ]
 [-3.7075043  -2.732352    0.24788637  2.7246304   1.0913134  -3.6348064 ]
 [-3.8298671  -2.886657   -0.74650145  2.3343172   2.0579534  -3.547966  ]
 [-3.7987633  -2.5885577   0.7180074   2.8261578   1.0192667  -3.776253  ]
 [-3.6875067  -2.5463917  -0.57753533  1.8462932   2.540622   -3.3265276 ]
 [-3.8163867  -2.839348   -0.7750374   2.164001    2.3274012  -3.5124736 ]
 [-3.8417878  -2.8785667  -0.6119624   2.43325     2.0008287  -3.6066515 ]
 [-4.1016774  -2.792976    0.07927184  2.5701303   2.0348547  -3.873624  ]
 [-3.5624173  -2.1825154   1.3866194   2.7420177   0.61436546 -3.5445993 ]
 [-3.5000863  -2.5638835  -0.9172853   1.6995534   2.393556   -3.0953698 ]
 [-3.891732   -2.923139   -0.34693173  2.5516047   1.956362   -3.6408195 ]
 [-3.890578   -2.261953    0.9011264   2.6127193   1.4754356  -3.8295465 ]
 [-3.59761    -2.2706008 

In [ ]:
print("Predictions 2 successfully completed.")

Predictions 2 successfully completed.
